In [5]:
import pandas as pd

all_data=pd.read_excel('data.xlsx') #Make sure the columns donot contain trailing spaces.
all_data

data=all_data[['Service Line','Service','SOP Reference','Task/Service/Reports','Effort','Frequency_Neat','Frequency','Multiplier','Monthly Hours']]
data.drop(data[data['Service'].isna()==True].index, inplace = True)

def get_nextmonth_date_details():
    import calendar
    import datetime

    current_month=datetime.datetime.today().month+1
    current_day=datetime.datetime.today().day
    current_year=datetime.datetime.today().year
    
    _, num_days = calendar.monthrange(current_year, current_month)
    first_day = datetime.date(current_year, current_month, 1)
    last_day = datetime.date(current_year, current_month, num_days)
    
    def workdays(d, end, excluded=(6, 7)):
        working_days=[]
        day_week=[]
        working_days_week_dic={}
        day_dict={1:'Monday',2:'Tuesday',3:'Wednesday',4:'Thursday',5:'Friday'}
        while d <= end:
            if d.isoweekday() not in excluded:
                working_days.append(str(d))
                working_days_week_dic[str(d)]=day_dict[d.isoweekday()]
                day_week.append(day_dict[d.isoweekday()])
            d += datetime.timedelta(days=1)
        return(working_days,working_days_week_dic,day_week)
    
    working_days,working_days_week_dic,day_week=workdays(first_day,last_day)
    
    ind=list(range(1,len(working_days)+1))
    working_days_dic=dict(zip(ind,working_days))
    ind_day_dic=dict(zip(ind,day_week))
    return(working_days_dic,working_days,working_days_week_dic,ind_day_dic)

def get_date_details():
    import calendar
    import datetime

    current_month=datetime.datetime.today().month
    current_day=datetime.datetime.today().day
    current_year=datetime.datetime.today().year
    
    _, num_days = calendar.monthrange(current_year, current_month)
    first_day = datetime.date(current_year, current_month, 1)
    last_day = datetime.date(current_year, current_month, num_days)
    
    def workdays(d, end, excluded=(6, 7)):
        working_days=[]
        day_week=[]
        working_days_week_dic={}
        day_dict={1:'Monday',2:'Tuesday',3:'Wednesday',4:'Thursday',5:'Friday'}
        while d <= end:
            if d.isoweekday() not in excluded:
                working_days.append(str(d))
                working_days_week_dic[str(d)]=day_dict[d.isoweekday()]
                day_week.append(day_dict[d.isoweekday()])
            d += datetime.timedelta(days=1)
        return(working_days,working_days_week_dic,day_week)
    
    working_days,working_days_week_dic,day_week=workdays(first_day,last_day)
    
    ind=list(range(1,len(working_days)+1))
    working_days_dic=dict(zip(ind,working_days))
    ind_day_dic=dict(zip(ind,day_week))
    return(working_days_dic,working_days,working_days_week_dic,ind_day_dic)

working_days_dic,working_days,working_days_week_dic,ind_day_dic=get_date_details()

def get_week_day_dates(working_days_week_dic):
    keys_list,values_list=[],[]
    for i in working_days_week_dic.keys():
        keys_list.append(i)
        values_list.append(working_days_week_dic[i])
        
    monday_indlist=[]
    tuesday_indlist=[]
    wednesday_indlist=[]
    thursday_indlist=[]
    friday_indlist=[]
    for ind,i in enumerate(values_list):
        if i =='Monday':
            monday_indlist.append(ind+1)
        elif i == 'Tuesday':
            tuesday_indlist.append(ind+1)
        elif i == 'Wednesday':
            wednesday_indlist.append(ind+1)
        elif i == 'Thursday':
            thursday_indlist.append(ind+1)
        elif i == 'Friday':
            friday_indlist.append(ind+1)
    #print(monday_indlist,tuesday_indlist,wednesday_indlist,thursday_indlist,friday_indlist)
    total_list=[monday_indlist,tuesday_indlist,wednesday_indlist,thursday_indlist,friday_indlist]
    monday_list=[]
    tuesday_list=[]
    wednesday_list=[]
    thursday_list=[]
    friday_list=[]
    for ind,i in enumerate(total_list):
        #print(ind)
        for j in i:
            #print(keys_list[j-1])
            if ind ==0:
                monday_list.append(keys_list[j-1])
            elif ind == 1:
                tuesday_list.append(keys_list[j-1])
            elif ind == 2:
                wednesday_list.append(keys_list[j-1])
            elif ind == 3:
                thursday_list.append(keys_list[j-1])
            elif ind == 4:
                friday_list.append(keys_list[j-1])
    #print(monday_list,tuesday_list,wednesday_list,thursday_list,friday_list)
    week_day_dict={'Monday':monday_list,
                 'Tuesday':tuesday_list,
                 'Wednesday':wednesday_list,
                  'Thursday':thursday_list,
                  'Friday':friday_list
                 }
    return(week_day_dict)
week_day_dict=get_week_day_dates(working_days_week_dic)
count={
'Daily':len(working_days),
'Monday':len(week_day_dict['Monday']),
'Tuesday':len(week_day_dict['Tuesday']),
'Wednesday':len(week_day_dict['Wednesday']),
'Thursday':len(week_day_dict['Thursday']),
'Friday':len(week_day_dict['Friday'])}

df = pd.DataFrame(columns = data.columns) 
#Values should not contain X+10 or X+9 like that. only one single value.
next_working_days_dic,next_working_days,next_working_days_week_dic,next_ind_day_dic=get_nextmonth_date_details()

for i in data.iterrows():
    #print(i[1]['Frequency_Neat'])
    try:
        if i[1]['Frequency_Neat']=='Daily':
            for j in range(count[i[1]['Frequency_Neat']]):
                #print(j,"--",i[0])
                srs= pd.Series(data.iloc[i[0]])
                srs['Date']=working_days_dic[j+1]
                srs=srs.to_frame().T
                df = df.append(srs)
                
        elif i[1]['Frequency_Neat'] in ['Monday','Tuesday','Wednesday','Thursday','Friday']:
             for j in range(count[i[1]['Frequency_Neat']]):
                #print(j,"--",i[0])
                for p,q in zip(['Monday','Tuesday','Wednesday','Thursday','Friday'],[j for i,j in week_day_dict.items()]):
                    if i[1]['Frequency_Neat'] ==p:
                        srs= pd.Series(data.iloc[i[0]])
                        for k_ind,k in enumerate(q):
                            if j==k_ind:
                                srs['Date']=k
                                srs=srs.to_frame().T
                                df = df.append(srs)
        else:
            if ('+' in i[1]['Frequency_Neat']):
                m,_,_,_=get_date_details()
                if int(i[1]['Frequency_Neat'].split('+')[1]) > max(m.keys()):
                    diff=int(i[1]['Frequency_Neat'].split('+')[1])-max(m.keys())
                    next_working_days_dic,_,_,_=get_nextmonth_date_details()
                    srs= pd.Series(data.iloc[i[0]])
                    srs['Date']=next_working_days_dic[diff]
                    srs=srs.to_frame().T
                    df = df.append(srs)                
                
                elif i[1]['Frequency_Neat'].split('+')[1] != '0':
                    #print(i[1]['Frequency_Neat'].split('+')[1],'  ',working_days_dic[int(i[1]['Frequency_Neat'].split('+')[1])])
                    srs= pd.Series(data.iloc[i[0]])
                    srs['Date']=working_days_dic[int(i[1]['Frequency_Neat'].split('+')[1])]
                    srs=srs.to_frame().T
                    df = df.append(srs)
                elif i[1]['Frequency_Neat'].split('+')[1] == '0':
                    #print(i[1]['Frequency_Neat'].split('+')[1])
                    srs= pd.Series(data.iloc[i[0]])
                    srs['Date']='Adhoc'
                    srs=srs.to_frame().T
                    df = df.append(srs)
                
                
                
                    
            elif ('-' in i[1]['Frequency_Neat']):
                #print("-----",i[1]['Frequency_Neat'].split('-')[1],'=======',str(max(a.keys())-int(i[1]['Frequency_Neat'].split('-')[1])))
                a,_,_,_=get_nextmonth_date_details()
                
                srs= pd.Series(data.iloc[i[0]])
                srs['Date']=working_days_dic[int(max(a.keys())-int(i[1]['Frequency_Neat'].split('-')[1]))]
                srs=srs.to_frame().T
                df = df.append(srs)
        
    except:
        print("exception")
        raise
df.reset_index(drop=True,inplace=True) 




def get_task_details(df,val):    
    temp_df=df.copy()
    temp_df['New Tasks'] = df['Service'].str.cat(df['SOP Reference'].fillna('No SOP'),sep="-->").str.cat(df['Task/Service/Reports'],sep="-->").str.cat(df['Frequency'],sep="-->").copy()
    task_details=""
    for i,j in temp_df[temp_df['Date'].isin([val])].filter(items=['New Tasks']).items():
        for k in j:
            task_details+=k
            task_details+='\n'
    return(task_details)



def get_task(df,val):
    temp_df1=df.copy()
    tasks=""
    for i,j in temp_df1[temp_df1['Date'].isin([val])].filter(items=['Task/Service/Reports']).items():
        for k in j:
            #print(k)
            tasks+=k
            tasks+='\n'
    return(tasks)
    
def send_email(body, subject, recipient,signature, auto=True):
    import win32com.client as win32   

    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = recipient
    mail.Subject = subject
    
    body=body+'<br>'*1+signature
    mail.HtmlBody = body
    if auto:
        mail.send
    else:
        mail.open 
def show_draft(body, subject, recipient,signature, auto=True):
    import win32com.client as win32   

    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = recipient
    mail.Subject = subject
    mail.HtmlBody = body
    
    body=body+'<br>'*1+signature
    mail.HtmlBody = body
    if auto:
        mail.display()
    else:
        mail.open

resources_df=pd.read_excel('resources_list.xlsx')
resources_df
master_service_set=set()
for i,j in resources_df['Service'].items():
    for k in j.split(','):
        master_service_set.add(k)
mail_dic={}
for i,j in resources_df['Service'].items():
    for k in j.split(','):
        if k in list(master_service_set):
            if len(mail_dic)==0:
                mail_dic[k]=[resources_df['Email'][i]]
                #print(mail_dic)
                if len(mail_dic[k])==0:
                    mail_dic[k]=[resources_df['Email'][i]]
            else:
                #print("Dic",mail_dic)
                try:
                    if len(mail_dic[k])==0:
                        mail_dic[k]=[resources_df['Email'][i]]
                    else:
                        mail_dic[k].append(resources_df['Email'][i])
                except:
                    mail_dic[k]=[resources_df['Email'][i]]
                
print('')
          
import PySimpleGUI as sg
import sys
import os
import docx
import time
import win32com.client as win32   

import time
named_tuple = time.localtime()
time_string=time.strftime("%Y-%m-%d",named_tuple)
time_string

sg.theme('DarkBlue17') 

layout = [ 
          [sg.Text('Task Reminder', size=(36, 1), justification='center', font=("Helvetica",20), relief=sg.RELIEF_RIDGE)],    
         
    [sg.Text('Service Line Name: '),sg.InputCombo(list(df['Service Line'].unique()),key='Service Line', size=(20, 2), enable_events=True),
      sg.Text('Service Name: '),sg.InputText('', key='Service',size=(20, 10),enable_events=True)],   
    
    [sg.Text('Date List:', pad=((60,10),(0,0))), (sg.InputCombo(working_days,key='date2', size=(20, 20), enable_events=True)),
                sg.Text('Date:', pad=((61,0),(0,0))),sg.InputText(time_string, key='date',size=(20, 10), pad=((10,50),(0,0)),enable_events=True)],
    
    [sg.Button('Show Details',size=(20,1)),sg.Button('Show Tasks',size=(20,1)),sg.Text('All Tasks:', pad=((20,10),(0,0))),
    sg.InputCombo(list(df['Service Line'].unique()),key='All_Service_Line', size=(20, 2), enable_events=True)],
    
    [sg.Text('Tasks-')],[ sg.Multiline('', key='tasks',size=(80, 25))],       
            
    [sg.Button('Send Mail'),sg.Button('Show Draft'),sg.Button('Update')]
         ]
updated_count=0
window = sg.Window('Remind Me',resizable=True).Layout(layout)
while True:
    event, values = window.read()
    if event == 'Show Tasks':
        if (len(values['Service']) ==0) and (len(values['All_Service_Line'])==0):
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        elif (len(values['Service']) > 0) and (len(values['All_Service_Line'])==0):
            window['tasks'].update(get_task(df[df['Service']==values['Service']],values['date']))
        elif len(values['All_Service_Line'])>0:
            window['tasks'].update(get_task(df[df['Service Line']==values['All_Service_Line']],values['date']))        
        
    if event == 'Show Details':        
        if (len(values['Service']) ==0) and (len(values['All_Service_Line'])==0):
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        elif (len(values['Service']) > 0) and (len(values['All_Service_Line'])==0):
            window['tasks'].update(get_task_details(df[df['Service']==values['Service']],values['date']))
        if len(values['All_Service_Line'])>0:
            window['tasks'].update(get_task_details(df[df['Service Line']==values['All_Service_Line']],values['date']))

    if event == 'Service Line':
        layout2=[ [sg.Text('Service Name: ')
                 ,sg.InputCombo(list(df[df['Service Line']==values['Service Line']]['Service'].unique()),key='Service2', size=(20, 10), enable_events=True)]]
        window2=sg.Window('Choose Service Name',resizable=True).Layout(layout2)
        event2, values2 = window2.read()
        if event2 == 'Service2':
            window['Service'].update(values2['Service2'])
            window2.close()
        window['All_Service_Line'].update('') 
        
    if event == 'date2':
        window['date'].update(values['date2'])
    
    if event == 'All_Service_Line':
        window['Service'].update('')
        window['Service Line'].update('')
        
    if event == 'Show Task Details':
        window['tasks'].update(get_task_details(df[df['Service']==values['Service']],values['date']))
        
        if event2 == 'Service2':
            window['Service'].update(values2['Service2'])
            window2.close()
        window['tasks'].update(get_task_details(df[df['Service Line']==values['All_Service_Line']],values['date']))
    
    if event == 'Show Draft':
        if len(values['Service']) <3:
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        else:
            body="""Hello All,
    
            Please make sure the tasks for """+time_string+""" are completed by EOD. \nTasks for the day are :\n\n"""+get_task(df[df['Service']==values['Service']],values['date'])
            
            subject="Reminder for Tasks "+time_string+" for "+values['Service']+"."
            
            signature="Regards,<br>Asset Management Team"
            recipient=";".join(mail_dic[values['Service']])+";VK00489900@techmahindra.com"
            show_draft(body.replace('\n','<br>'), subject, recipient,signature, auto=True)
    if event == 'Send Mail':
        if len(values['Service']) <3:
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        else:
            body="""Hello All,
    
            Please make sure the tasks for """+time_string+""" are completed by EOD. \nTasks for the day are :\n\n"""+get_task(df[df['Service']==values['Service']],values['date'])
            
            subject="Reminder for Tasks "+time_string+" for "+values['Service']+"."
            
            signature="Regards,<br>Asset Management Team"
            recipient=";".join(mail_dic[values['Service']])+";VK00489900@techmahindra.com"
            send_email(body.replace('\n','<br>'), subject, recipient,signature, auto=True)
    
    if event == 'Update':
        if (len(values['Service']) ==0) and (len(values['All_Service_Line'])==0):
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        else:
            def checkbox_data(service,date): 
                bf=df[df['Service']==service].copy()
                a=[]
                for i,j in bf[bf['Date'].isin([date])].filter(items=['Task/Service/Reports']).items():
                    for k in j:
                        a+=[sg.Checkbox(k,key=k)]
                col=[a]
                b=[]
                for i in col:
                    for j in i:
                        b.append([j])
                return(b)
            
            check_items=checkbox_data(values['Service'],values['date'])
            
            layout4 = [[ sg.Column(check_items,scrollable=True)], [sg.Button('OK')]]
            window4 = sg.Window('Task Check Boxes ',resizable=True).Layout(layout4)
            
            if updated_count==1:
                sg.popup('Looks like you have already updated the tasks.')
                updated_count=0
                
            event4, values4 = window4.read()
            
            try:
                
                if os.path.isfile('Checked_data.xlsx'):
                    #checked_df=pd.read_excel('Checked_data.xlsx')
                    new_checked_df= pd.DataFrame(values4, index=['Status']).T.reset_index()
                    new_checked_df.to_excel('Checked_data.xlsx')
                else:
                    checked_df= pd.DataFrame(values4, index=['Status']).T.reset_index()
                    checked_df.to_excel('Checked_data.xlsx',index=0)
                
           
                
                if event4 =='OK':
                    window4.close()
                    
            except:
                sg.popup('Make Sure you closed the Checked_data excel.')
                raise
            
                
       
    if event == sg.WIN_CLOSED:
        break         

In [2]:
     
import PySimpleGUI as sg
import sys
import os
import docx
import time
import win32com.client as win32   

import time
named_tuple = time.localtime()
time_string=time.strftime("%Y-%m-%d",named_tuple)
time_string

sg.theme('DarkBlue17') 

layout = [ 
          [sg.Text('Task Reminder', size=(36, 1), justification='center', font=("Helvetica",20), relief=sg.RELIEF_RIDGE)],    
         
    [sg.Text('Service Line Name: '),sg.InputCombo(list(df['Service Line'].unique()),key='Service Line', size=(20, 2), enable_events=True),
      sg.Text('Service Name: '),sg.InputText('', key='Service',size=(20, 10),enable_events=True)],   
    
    [sg.Text('Date List:', pad=((60,10),(0,0))), (sg.InputCombo(working_days,key='date2', size=(20, 20), enable_events=True)),
                sg.Text('Date:', pad=((61,0),(0,0))),sg.InputText(time_string, key='date',size=(20, 10), pad=((10,50),(0,0)),enable_events=True)],
    
    [sg.Button('Show Details',size=(20,1)),sg.Button('Show Tasks',size=(20,1)),sg.Text('All Tasks:', pad=((20,10),(0,0))),
    sg.InputCombo(list(df['Service Line'].unique()),key='All_Service_Line', size=(20, 2), enable_events=True)],
    
    [sg.Text('Tasks-')],[ sg.Multiline('', key='tasks',size=(80, 25))],       
            
    [sg.Button('Send Mail'),sg.Button('Show Draft')]
         ]

window = sg.Window('Remind Me',resizable=True).Layout(layout)
while True:
    event, values = window.read()
    if event == 'Show Tasks':
        if (len(values['Service']) <1) and (len(values['All_Service_Line'])==0):
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        elif (len(values['Service']) > 1) and (len(values['All_Service_Line'])==0):
            window['tasks'].update(get_task(df[df['Service']==values['Service']],values['date']))
        if len(values['All_Service_Line'])>0:
            window['tasks'].update(get_task(df[df['Service Line']==values['All_Service_Line']],values['date']))
        
        
    if event == 'Show Details':
        
        if (len(values['Service']) <1) and (len(values['All_Service_Line'])==0):
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        elif (len(values['Service']) > 1) and (len(values['All_Service_Line'])==0):
            window['tasks'].update(get_task_details(df[df['Service']==values['Service']],values['date']))
        if len(values['All_Service_Line'])>0:
            window['tasks'].update(get_task_details(df[df['Service Line']==values['All_Service_Line']],values['date']))

    if event == 'Service Line':
        layout2=[ [sg.Text('Service Name: ')
                 ,sg.InputCombo(list(df[df['Service Line']==values['Service Line']]['Service'].unique()),key='Service2', size=(20, 10), enable_events=True)]]
        window2=sg.Window('Choose Service Name',resizable=True).Layout(layout2)
        event2, values2 = window2.read()
        if event2 == 'Service2':
            window['Service'].update(values2['Service2'])
            window2.close()
        window['All_Service_Line'].update('') 
        
    if event == 'date2':
        window['date'].update(values['date2'])
    
    if event == 'All_Service_Line':
        window['Service'].update('')
        
    if event == 'Show Task Details':
        window['tasks'].update(get_task_details(df[df['Service']==values['Service']],values['date']))
        
        if event2 == 'Service2':
            window['Service'].update(values2['Service2'])
            window2.close()
        window['tasks'].update(get_task_details(df[df['Service Line']==values['All_Service_Line']],values['date']))
    
    if event == 'Show Draft':
        if len(values['Service']) <3:
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        else:
            body="""Hello All,
    
            Please make sure the tasks for """+time_string+""" are completed by EOD. \nTasks for the day are :\n\n"""+get_task(df[df['Service']==values['Service']],values['date'])
            
            subject="Reminder for Tasks "+time_string+" for "+values['Service']+"."
            
            signature="Regards,<br>Asset Management Team"
            recipient=";".join(mail_dic[values['Service']])+";VK00489900@techmahindra.com"
            show_draft(body.replace('\n','<br>'), subject, recipient,signature, auto=True)
    if event == 'Send Mail':
        if len(values['Service']) <3:
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        else:
            body="""Hello All,
    
            Please make sure the tasks for """+time_string+""" are completed by EOD. \nTasks for the day are :\n\n"""+get_task(df[df['Service']==values['Service']],values['date'])
            
            subject="Reminder for Tasks "+time_string+" for "+values['Service']+"."
            
            signature="Regards,<br>Asset Management Team"
            recipient=";".join(mail_dic[values['Service']])+";VK00489900@techmahindra.com"
            send_email(body.replace('\n','<br>'), subject, recipient,signature, auto=True)
        
        
    if event == sg.WIN_CLOSED:
        break


In [6]:
          
import PySimpleGUI as sg
import sys
import os
import docx
import time
import win32com.client as win32   

import time
named_tuple = time.localtime()
time_string=time.strftime("%Y-%m-%d",named_tuple)
time_string

sg.theme('DarkBlue17') 

layout = [ 
          [sg.Text('Task Reminder', size=(36, 1), justification='center', font=("Helvetica",20), relief=sg.RELIEF_RIDGE)],    
         
    [sg.Text('Service Line Name: '),sg.InputCombo(list(df['Service Line'].unique()),key='Service Line', size=(20, 2), enable_events=True),
      sg.Text('Service Name: '),sg.InputText('', key='Service',size=(20, 10),enable_events=True)],   
    
    [sg.Text('Date List:', pad=((60,10),(0,0))), (sg.InputCombo(working_days,key='date2', size=(20, 20), enable_events=True)),
                sg.Text('Date:', pad=((61,0),(0,0))),sg.InputText(time_string, key='date',size=(20, 10), pad=((10,50),(0,0)),enable_events=True)],
    
    [sg.Button('Show Details',size=(20,1)),sg.Button('Show Tasks',size=(20,1)),sg.Text('All Tasks:', pad=((20,10),(0,0))),
    sg.InputCombo(list(df['Service Line'].unique()),key='All_Service_Line', size=(20, 2), enable_events=True)],
    
    [sg.Text('Tasks-')],[ sg.Multiline('', key='tasks',size=(80, 25))],       
            
    [sg.Button('Send Mail'),sg.Button('Show Draft'),sg.Button('Update')]
         ]
updated_count=0
window = sg.Window('Remind Me',resizable=True).Layout(layout)
while True:
    event, values = window.read()
    if event == 'Show Tasks':
        if (len(values['Service']) ==0) and (len(values['All_Service_Line'])==0):
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        elif (len(values['Service']) > 0) and (len(values['All_Service_Line'])==0):
            window['tasks'].update(get_task(df[df['Service']==values['Service']],values['date']))
        elif len(values['All_Service_Line'])>0:
            window['tasks'].update(get_task(df[df['Service Line']==values['All_Service_Line']],values['date']))        
        
    if event == 'Show Details':        
        if (len(values['Service']) ==0) and (len(values['All_Service_Line'])==0):
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        elif (len(values['Service']) > 0) and (len(values['All_Service_Line'])==0):
            window['tasks'].update(get_task_details(df[df['Service']==values['Service']],values['date']))
        if len(values['All_Service_Line'])>0:
            window['tasks'].update(get_task_details(df[df['Service Line']==values['All_Service_Line']],values['date']))

    if event == 'Service Line':
        layout2=[ [sg.Text('Service Name: ')
                 ,sg.InputCombo(list(df[df['Service Line']==values['Service Line']]['Service'].unique()),key='Service2', size=(20, 10), enable_events=True)]]
        window2=sg.Window('Choose Service Name',resizable=True).Layout(layout2)
        event2, values2 = window2.read()
        if event2 == 'Service2':
            window['Service'].update(values2['Service2'])
            window2.close()
        window['All_Service_Line'].update('') 
        
    if event == 'date2':
        window['date'].update(values['date2'])
    
    if event == 'All_Service_Line':
        window['Service'].update('')
        window['Service Line'].update('')
        
    if event == 'Show Task Details':
        window['tasks'].update(get_task_details(df[df['Service']==values['Service']],values['date']))
        
        if event2 == 'Service2':
            window['Service'].update(values2['Service2'])
            window2.close()
        window['tasks'].update(get_task_details(df[df['Service Line']==values['All_Service_Line']],values['date']))
    
    if event == 'Show Draft':
        if len(values['Service']) <3:
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        else:
            body="""Hello All,
    
            Please make sure the tasks for """+time_string+""" are completed by EOD. \nTasks for the day are :\n\n"""+get_task(df[df['Service']==values['Service']],values['date'])
            
            subject="Reminder for Tasks "+time_string+" for "+values['Service']+"."
            
            signature="Regards,<br>Asset Management Team"
            recipient=";".join(mail_dic[values['Service']])+";VK00489900@techmahindra.com"
            show_draft(body.replace('\n','<br>'), subject, recipient,signature, auto=True)
    if event == 'Send Mail':
        if len(values['Service']) <3:
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        else:
            body="""Hello All,
    
            Please make sure the tasks for """+time_string+""" are completed by EOD. \nTasks for the day are :\n\n"""+get_task(df[df['Service']==values['Service']],values['date'])
            
            subject="Reminder for Tasks "+time_string+" for "+values['Service']+"."
            
            signature="Regards,<br>Asset Management Team"
            recipient=";".join(mail_dic[values['Service']])+";VK00489900@techmahindra.com"
            send_email(body.replace('\n','<br>'), subject, recipient,signature, auto=True)
    
    if event == 'Update':
        if (len(values['Service']) ==0) and (len(values['All_Service_Line'])==0):
            sg.popup("Alert","Please enter a service value before looking for tasks.")
        else:
            def checkbox_data(service,date): 
                bf=df[df['Service']==service].copy()
                a=[]
                for i,j in bf[bf['Date'].isin([date])].filter(items=['Task/Service/Reports']).items():
                    for k in j:
                        a+=[sg.Checkbox(k,key=k)]
                col=[a]
                b=[]
                for i in col:
                    for j in i:
                        b.append([j])
                return(b)
            
            check_items=checkbox_data(values['Service'],values['date'])
            
            layout4 = [[ sg.Column(check_items,scrollable=True)], [sg.Button('Check'),sg.Button('Confirm'),sg.Button('Save and Close')]]
            window4 = sg.Window('Task Check Boxes ',resizable=True).Layout(layout4)
            
            if updated_count==1:
                sg.popup('Looks like you have already updated the tasks.')
                updated_count=0
            
            while True:
                event4, values4 = window4.read()
                
                try:

                    if event4 == 'Check':                       
                        if os.path.isfile('Checked_data.xlsx'):
                                
                            test_df=pd.DataFrame(values4,index=['Status']).T.reset_index() 
                            checked_df=pd.read_excel('Checked_data.xlsx')
                            
                            l1=list(checked_df['index'])
                            l2=list(test_df['index'])

                            for i_ind,i in enumerate(l1):
                                for j_ind,j in enumerate(l2):
                                    if i == j :
                                        if checked_df['Status'][i_ind]==True:
                                            window4[i].Update(disabled=True)
                            del test_df  
                            del checked_df
                        else:
                            sg.popup('There is no saved data to perform a check.')
                        
                       
                    if event4 =='Confirm': 

                        def confirm():
                            if os.path.isfile('Checked_data.xlsx'):
                                checked_df= pd.read_excel('Checked_data.xlsx')  
                                test_df=pd.DataFrame(values4,index=['Status']).T.reset_index()
                                
                                filter_cols=['index','Status']
                                checked_df.reset_index(drop=True,inplace=True)
                                checked_df=checked_df.filter(items=filter_cols)
                                #checked_df=checked_df.append(test_df)
                                #checked_df = checked_df.drop_duplicates(subset='index', keep="last")
                                checked_df.reset_index(drop=True,inplace=True)
                                
                            else:
                                checked_df= pd.DataFrame(values4, index=['Status']).T.reset_index() #revisit
                                checked_df.reset_index(drop=True,inplace=True)
                                test_df=pd.DataFrame(values4,index=['Status']).T.reset_index()
                        

                            l1=list(checked_df['index'])
                            l2=list(test_df['index'])
                            test_df=pd.DataFrame(values4,index=['Status']).T.reset_index()
                            count=0
                            temp_list_index=[]
                            temp_list_status=[]
                            for j_ind,j in enumerate(l2):
                                for i_ind,i in enumerate(l1):
                                    if i == j :
                                        #print(i_ind,j_ind,checked_df,test_df)
                                        if checked_df['Status'][i_ind]==False and test_df['Status'][j_ind] == True:
                                            checked_df['Status'][i_ind]=test_df['Status'][j_ind]
                                        if checked_df['Status'][i_ind]==True:
                                            checked_df['Status'][i_ind]=True
                                        if checked_df['Status'][i_ind] not in [True,False]:
                                            checked_df['Status'][i_ind]=False
                                    else:
                                        if test_df['Status'][j_ind] == True:
                                            
                                            temp_list_index.append(j)
                                            temp_list_status.append(True)
                            temp_dic={'index':list(set(temp_list_index)),'Status':[True for i in range(len(set(list(temp_list_index))))]}
                            #print(temp_dic,checked_df)
                            
                            #checked_df.append(pd.DataFrame(temp_dic).T,ignore_index=True)
                            checked_df=pd.DataFrame(temp_dic).append(checked_df,ignore_index=True)
                           
                            #checked_df.append(temp_df).reset_index(drop=True,inplace=True)
                                                                                                    
                         
                            filter_cols=['index','Status']
                            checked_df=checked_df.filter(items=filter_cols)
                            checked_df.append(test_df).reset_index(drop=True)
                            checked_df = checked_df.drop_duplicates(subset='index', keep="last")
                            checked_df.to_excel('Checked_data.xlsx', index=0)
                            
                            def get_check_data_details():
                                checked_df = pd.read_excel('Checked_data.xlsx')
                                checked_df = checked_df.reindex(columns=['Service Line', 'Service', 'index', 'Status', 'Task Due Date', 'Task Updated Date'])        
                            
                                for i_ind,i in enumerate(checked_df['index']):
                                    if len(list(df[df['Task/Service/Reports']==i]['Service'].unique()))>1:
                                        sg.popup('Looks like a same task is present in 2 different Services. Please check the task name.')
                                    else: 
                                        checked_df['Service'][i_ind]=list(df[df['Task/Service/Reports']==i]['Service'].unique())[0]
                                        checked_df['Service Line'][i_ind]=list(df[df['Task/Service/Reports']==i]['Service Line'].unique())[0]
                                        checked_df['Task Due Date'][i_ind]=values['date']
                                        checked_df['Task Updated Date'][i_ind]=time_string
                                                                
                                return(checked_df)
                            test_checked_df=checked_df.copy()
                            test_checked_df.to_excel('test_checked_df.xlsx')
                            checked_df=get_check_data_details()
                            checked_df = checked_df.drop_duplicates(subset='index', keep="last")
                            checked_df.to_excel('Checked_data.xlsx', index=0)
                            
                        confirm()    
                        for i,j in values4.items():
                            if i in list(values4.keys()):
                                if values4[i]==True:
                                    window4[i].Update(disabled=True)
                           
                        
                    if event4 == sg.WIN_CLOSED:
                        break    
                    
                    if event4 =='Save and Close':
                        window4.close()
                        confirm()
                                                
                except:
                    sg.popup('Make Sure you closed the Checked_data excel.')
                    #raise
                    break
                    
                    
            
                
       
    if event == sg.WIN_CLOSED:
        break

<ipython-input-6-25de312acce3>:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checked_df['Status'][i_ind]=True
<ipython-input-6-25de312acce3>:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checked_df['Service'][i_ind]=list(df[df['Task/Service/Reports']==i]['Service'].unique())[0]
<ipython-input-6-25de312acce3>:219: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checked_df['Service Line'][i_ind]=list(df[df['Task/Service/Reports']

In [10]:
test_dic={'index': ['', 'Quarterly Review - San Diego', 'Quarterly Review - Europe', 'Quarterly Review - Taiwan'], 'Status': [True, True, True, True, True],'date':}  

In [20]:
checked_df.shape[0]

NameError: name 'checked_df' is not defined

In [43]:
dic={'Allianz GI Hong Kong Reporting - Daily Pair': True, 'US Daily Reports': True, 'Miscellanous Reports - 2Sigma Reporting (ProUI & Luna)': True, 'Risk Reports - VaR – daily delivery to external managers': True, 'Risk Reports - Controlling Volatility Strategy': False, 'Risk Reports - Dynamic Risk Parity': False, 'Risk Reports - Allianz Fundamental Strategy LUX': False, 'Risk Reports - Cash Flow Report EU': False, 'Risk Reports - Daily Liquidity Risk Report (AP)': False, 'Risk Reports - Daily Liquidity Risk Report (US)': False, 'Risk Reports - Merger Arbitrage': False, 'Risk Reports - Allianz Fixed Income Macro': False, 'Risk Reports - VaR-Monitoring MAS (EU)': False, 'Risk Reports - VaR-Monitoring MAS (AP)': False, 'Risk Reports - VaR-Monitoring MAS (US)': False, 'Risk Reports - VaR-Monitoring FI': False, 'Risk Reports - Var Monitoring VR-Bank': False, 'Risk Reports - Var Monitoring Suscap': False, 'Risk Reports - VaR – weekly delivery to external managers': False, 'Jupiter Reporting': False}

In [55]:
checked_df
temp_dic={'index':'Rajaa','Status':True}

temp_df=pd.DataFrame(temp_dic, index=[0]).append(checked_df)
temp_dic={'index':'vamsi','Status':True}

temp_df=pd.DataFrame(temp_dic, index=[0]).append(checked_df)
temp_df

index  Status  \
0                                               vamsi    True   
0         Allianz GI Hong Kong Reporting - Daily Pair    True   
1                                    US Daily Reports    True   
2   Miscellanous Reports - 2Sigma Reporting (ProUI...    True   
3   Risk Reports - VaR – daily delivery to externa...    True   
4      Risk Reports - Controlling Volatility Strategy   False   
5                  Risk Reports - Dynamic Risk Parity   False   
6     Risk Reports - Allianz Fundamental Strategy LUX    True   
7                  Risk Reports - Cash Flow Report EU    True   
8     Risk Reports - Daily Liquidity Risk Report (AP)    True   
9     Risk Reports - Daily Liquidity Risk Report (US)   False   
10                    Risk Reports - Merger Arbitrage   False   
11          Risk Reports - Allianz Fixed Income Macro   False   
12             Risk Reports - VaR-Monitoring MAS (EU)   False   
13             Risk Reports - VaR-Monitoring MAS (AP)   False   
14             Risk Reports - VaR-Monitoring MAS (US)    True   
15                   Risk Reports - VaR-Monitoring FI    True   
16              Risk Reports - Var Monitoring VR-Bank    True   
17               Risk Reports - Var Monitoring Suscap    True   
18  Risk Reports - VaR – weekly delivery to extern...    True   
19                                  Jupiter Reporting    True   

        Service Line               Service Task Due Date Task Updated Date  
0                NaN                   NaN           NaN               NaN  
0   Asset Management  Customized Reporting    2020-09-01        2020-09-26  
1   Asset Management  Customized Reporting    2020-09-01        2020-09-26  
2   Asset Management  Customized Reporting    2020-09-01        2020-09-26  
3   Asset Management  Customized Reporting    2020-09-01        2020-09-26  
4   Asset Management  Customized Reporting    2020-09-01        2020-09-26  
5   Asset Management  Customized Reporting    2020-09-01        2020-09-26  
6   Asset Management  Customized Reporting    2020-09-01        2020-09-26  
7   Asset Management  Customized Reporting    2020-09-01        2020-09-26  
8   Asset Management  Customized Reporting    2020-09-01        2020-09-26  
9   Asset Management  Customized Reporting    2020-09-01        2020-09-26  
10  Asset Management  Customized Reporting    2020-09-01        2020-09-26  
11  Asset Management  Customized Reporting    2020-09-01        2020-09-26  
12  Asset Management  Customized Reporting    2020-09-01        2020-09-26  
13  Asset Management  Customized Reporting    2020-09-01        2020-09-26  
14  Asset Management  Customized Reporting    2020-09-01        2020-09-26  
15  Asset Management  Customized Reporting    2020-09-01        2020-09-26  
16  Asset Management  Customized Reporting    2020-09-01        2020-09-26  
17  Asset Management  Customized Reporting    2020-09-01        2020-09-26  
18  Asset Management  Customized Reporting    2020-09-01        2020-09-26  
19  Asset Management  Customized Reporting    2020-09-01        2020-09-26

In [5]:

import PySimpleGUI as sg

# Demo of how columns work
# window has on row 1 a vertical slider followed by a COLUMN with 7 rows
# Prior to the Column element, this layout was not possible
# Columns layouts look identical to window layouts, they are a list of lists of elements.

window = sg.Window('Columns',size=(100,100))                                   # blank window

# Column layout
col1 = [[sg.Text('col Row 1')],
       [sg.Text('col Row 2'), sg.Input('col input 1')],
       [sg.Text('col Row 3'), sg.Input('col input 2')],
       [sg.Text('col Row 4'), sg.Input('col input 3')],
       [sg.Text('col Row 5'), sg.Input('col input 4')],
       [sg.Text('col Row 6'), sg.Input('col input 5')],
       [sg.Text('col Row 7'), sg.Input('col input 6')]]

val='2020-09-16'
def test2(): 
    a=[]
    for i,j in df[df['Date'].isin([val])].filter(items=['Task/Service/Reports']).items():
        for k in j:
            a+=[sg.Checkbox(k,key=k)]
    col=[a]
    b=[]
    for i in col:
        for j in i:
            b.append([j])
    return(b)

def checkbox_data(service,date): 
    bf=df[df['Service']==service].copy()
    a=[]
    for i,j in bf[bf['Date'].isin([date])].filter(items=['Task/Service/Reports']).items():
        for k in j:
            a+=[sg.Checkbox(k,key=k)]
    col=[a]
    b=[]
    for i in col:
        for j in i:
            b.append([j])
    return(b)



layout5 = [[ sg.Column(checkbox_data('OPF','2020-09-16'),size=(300,300),scrollable=True)], [sg.OK()]]

# Display the window and get values

window5 = sg.Window('Compact 1-line window with column',resizable=True).Layout(layout5)
event5, values5 = window5.read()
window5.close()

sg.Popup(event5, values5, line_width=200)

'OK'

In [57]:
df[df['Task/Service/Reports']=='Allianz GI Hong Kong Reporting - Daily Pair']['Date']=='2020-09-28'


0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
Name: Date, dtype: bool

In [43]:
#checked_df.drop_duplicates(keep='last').reset_index(drop=True)

test_checked_df=pd.read_excel('test_checked_df.xlsx')
test_checked_df = test_checked_df.reindex(columns=['Service Line', 'Service', 'index', 'Status', 'Task Due Date', 'Task Updated Date'])        
for i_ind,i in enumerate(test_checked_df['index']):
    if len(list(df[df['Task/Service/Reports']==i]['Service'].unique()))>1:
        sg.popup('Looks like a same task is present in 2 different Services. Please check the task name.')
    else:
        if i in list(df['Task/Service/Reports']):
            test_checked_df['Service'][i_ind]=list(df[df['Task/Service/Reports']==i]['Service'].unique())[0]
            test_checked_df['Service Line'][i_ind]=list(df[df['Task/Service/Reports']==i]['Service Line'].unique())[0]
            test_checked_df['Task Due Date'][i_ind]='20-20-2020'
            test_checked_df['Task Updated Date'][i_ind]=time_string
#test_checked_df = test_checked_df.reindex(columns=['Service Line', 'Service', 'index', 'Status', 'Task Due Date', 'Task Updated Date'])        
test_checked_df

<ipython-input-43-b01518f7c391>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_checked_df['Service'][i_ind]=list(df[df['Task/Service/Reports']==i]['Service'].unique())[0]
C:\Users\vk00489900\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-43-b01518f7c391>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_checked

Service Line               Service  \
0   Asset Management                   OPF   
1   Asset Management                   OPF   
2   Asset Management  Customized Reporting   
3   Asset Management  Customized Reporting   
4   Asset Management  Customized Reporting   
5   Asset Management  Customized Reporting   
6   Asset Management  Customized Reporting   
7   Asset Management  Customized Reporting   
8   Asset Management  Customized Reporting   
9   Asset Management  Customized Reporting   
10  Asset Management  Customized Reporting   
11  Asset Management  Customized Reporting   
12  Asset Management  Customized Reporting   
13  Asset Management  Customized Reporting   
14  Asset Management  Customized Reporting   
15  Asset Management  Customized Reporting   
16  Asset Management  Customized Reporting   
17  Asset Management  Customized Reporting   
18  Asset Management  Customized Reporting   
19  Asset Management  Customized Reporting   
20  Asset Management  Customized Reporting   
21  Asset Management  Customized Reporting   
22  Asset Management  Customized Reporting   
23  Asset Management  Customized Reporting   
24  Asset Management  Customized Reporting   
25  Asset Management  Customized Reporting   
26  Asset Management  Customized Reporting   
27  Asset Management  Customized Reporting   
28  Asset Management  Customized Reporting   
29  Asset Management  Customized Reporting   
30  Asset Management  Customized Reporting   

                                                index  Status Task Due Date  \
0                                                opf1    True    20-20-2020   
1                                                opf1    True    20-20-2020   
2                                    US Daily Reports    True    20-20-2020   
3         Allianz GI Hong Kong Reporting - Daily Pair    True    20-20-2020   
4                                    US Daily Reports    True    20-20-2020   
5         Allianz GI Hong Kong Reporting - Daily Pair    True    20-20-2020   
6         Allianz GI Hong Kong Reporting - Daily Pair    True    20-20-2020   
7                                    US Daily Reports    True    20-20-2020   
8   Miscellanous Reports - 2Sigma Reporting (ProUI...   False    20-20-2020   
9   Risk Reports - VaR – daily delivery to externa...   False    20-20-2020   
10     Risk Reports - Controlling Volatility Strategy   False    20-20-2020   
11                 Risk Reports - Dynamic Risk Parity   False    20-20-2020   
12    Risk Reports - Allianz Fundamental Strategy LUX   False    20-20-2020   
13                 Risk Reports - Cash Flow Report EU   False    20-20-2020   
14    Risk Reports - Daily Liquidity Risk Report (AP)   False    20-20-2020   
15    Risk Reports - Daily Liquidity Risk Report (US)   False    20-20-2020   
16   Risk Reports - Weekly Stress Test Reporting (EU)   False    20-20-2020   
17                    Risk Reports - Merger Arbitrage   False    20-20-2020   
18          Risk Reports - Allianz Fixed Income Macro   False    20-20-2020   
19             Risk Reports - VaR-Monitoring MAS (EU)   False    20-20-2020   
20             Risk Reports - VaR-Monitoring MAS (AP)   False    20-20-2020   
21             Risk Reports - VaR-Monitoring MAS (US)   False    20-20-2020   
22                   Risk Reports - VaR-Monitoring FI   False    20-20-2020   
23              Risk Reports - Var Monitoring VR-Bank   False    20-20-2020   
24               Risk Reports - Var Monitoring Suscap   False    20-20-2020   
25                              Global Weekly reports   False    20-20-2020   
26                                      VAR reporting   False    20-20-2020   
27                                   Breakdown Report   False    20-20-2020   
28                                     Multi Asset AP   False    20-20-2020   
29               AGI US - After tax withholding taxes   False    20-20-2020   
30                       AGI US - Accrual fees for US   False    20-20-2020   

   Tas

In [10]:
for i_ind,i in enumerate(checked_df['index']):
    if len(list(df[df['Task/Service/Reports']=='US Daily Reports']['Service'].unique()))>1:
        sg.popup('Looks like a same task is present in 2 different Services. Please check the task name.')
    else:
        checked_df['Service'][i_ind]=list(df[df['Task/Service/Reports']=='US Daily Reports']['Service'].unique())[0]
        checked_df['Service Line'][i_ind]=list(df[df['Task/Service/Reports']=='US Daily Reports']['Service Line'].unique())[0]
        checked_df['Task Due Date'][i_ind]=time_string
        checked_df['Task Updated Date'][i_ind]=time_string
checked_df = checked_df.reindex(columns=['Service Line', 'Service', 'index', 'Status', 'Task Due Date', 'Task Updated Date'])        
checked_df

<ipython-input-10-a48ec75e9ab8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checked_df['Service'][i_ind]=list(df[df['Task/Service/Reports']=='US Daily Reports']['Service'].unique())[0]
<ipython-input-10-a48ec75e9ab8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checked_df['Service Line'][i_ind]=list(df[df['Task/Service/Reports']=='US Daily Reports']['Service Line'].unique())[0]
<ipython-input-10-a48ec75e9ab8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

Service Line               Service  \
0   Asset Management  Customized Reporting   
1   Asset Management  Customized Reporting   
2   Asset Management  Customized Reporting   
3   Asset Management  Customized Reporting   
4   Asset Management  Customized Reporting   
5   Asset Management  Customized Reporting   
6   Asset Management  Customized Reporting   
7   Asset Management  Customized Reporting   
8   Asset Management  Customized Reporting   
9   Asset Management  Customized Reporting   
10  Asset Management  Customized Reporting   
11  Asset Management  Customized Reporting   
12  Asset Management  Customized Reporting   
13  Asset Management  Customized Reporting   
14  Asset Management  Customized Reporting   
15  Asset Management  Customized Reporting   
16  Asset Management  Customized Reporting   
17  Asset Management  Customized Reporting   
18  Asset Management  Customized Reporting   
19  Asset Management  Customized Reporting   
20  Asset Management  Customized Reporting   
21  Asset Management  Customized Reporting   
22  Asset Management  Customized Reporting   
23  Asset Management  Customized Reporting   
24  Asset Management  Customized Reporting   
25  Asset Management  Customized Reporting   
26  Asset Management  Customized Reporting   
27  Asset Management  Customized Reporting   
28  Asset Management  Customized Reporting   
29  Asset Management  Customized Reporting   
30  Asset Management  Customized Reporting   
31  Asset Management  Customized Reporting   
32  Asset Management  Customized Reporting   
33  Asset Management  Customized Reporting   
34  Asset Management  Customized Reporting   

                                                index  Status Task Due Date  \
0                                                opf1    True    2020-09-28   
1                                                opf1    True    2020-09-28   
2                        AGI US - Accrual fees for US    True    2020-09-28   
3                                      Multi Asset AP    True    2020-09-28   
4                AGI US - After tax withholding taxes    True    2020-09-28   
5                        AGI US - Accrual fees for US    True    2020-09-28   
6                                      Multi Asset AP    True    2020-09-28   
7                AGI US - After tax withholding taxes    True    2020-09-28   
8                                    US Daily Reports    True    2020-09-28   
9         Allianz GI Hong Kong Reporting - Daily Pair    True    2020-09-28   
10        Allianz GI Hong Kong Reporting - Daily Pair    True    2020-09-28   
11                                   US Daily Reports    True    2020-09-28   
12  Miscellanous Reports - 2Sigma Reporting (ProUI...    True    2020-09-28   
13  Risk Reports - VaR – daily delivery to externa...    True    2020-09-28   
14     Risk Reports - Controlling Volatility Strategy   False    2020-09-28   
15                 Risk Reports - Dynamic Risk Parity   False    2020-09-28   
16    Risk Reports - Allianz Fundamental Strategy LUX   False    2020-09-28   
17                 Risk Reports - Cash Flow Report EU   False    2020-09-28   
18    Risk Reports - Daily Liquidity Risk Report (AP)   False    2020-09-28   
19    Risk Reports - Daily Liquidity Risk Report (US)   False    2020-09-28   
20   Risk Reports - Weekly Stress Test Reporting (EU)   False    2020-09-28   
21                    Risk Reports - Merger Arbitrage   False    2020-09-28   
22          Risk Reports - Allianz Fixed Income Macro   False    2020-09-28   
23             Risk Reports - VaR-Monitoring MAS (EU)   False    2020-09-28   
24             Risk Reports - VaR-Monitoring MAS (AP)   False    2020-09-28   
25             Risk Reports - VaR-Monitoring MAS (US)   False    2020-09-28   
26                   Risk Reports - VaR-Monitoring FI   False    2020-09-28   
27              Risk Reports - Var Monitoring VR-Bank   False    2020-09-28   
28               Risk Reports - Var Monitoring Suscap   Fals

In [5]:
checked_df=get_check_data_details()
list(checked_df.columns.values)
test_df=checked_df.filter(items=['index','Status'])

In [4]:
def get_check_data_details():
    checked_df = pd.read_excel('Checked_data.xlsx')
    for i in checked_df['index']:
        if len(list(df[df['Task/Service/Reports']=='US Daily Reports']['Service'].unique()))>1:
            sg.popup('Looks like a same task is present in 2 different Services. Please check the task name.')
        else:
            checked_df['Service']=list(df[df['Task/Service/Reports']=='US Daily Reports']['Service'].unique())[0]
            checked_df['Service Line']=list(df[df['Task/Service/Reports']=='US Daily Reports']['Service Line'].unique())[0]
            checked_df['Task Due Date']=list(df[df['Task/Service/Reports']=='US Daily Reports']['Date'].unique())[0]
            checked_df['Task Updated Date']=time_string
    checked_df = checked_df.reindex(columns=['Service Line', 'Service', 'index', 'Status', 'Task Due Date', 'Task Updated Date'])
    return(checked_df)


In [25]:


l1=list(checked_df['index'])
l2=list(test_df['index'])

for i_ind,i in enumerate(l1):
    for j_ind,j in enumerate(l2):
        if i == j :
            if checked_df['Status'][i_ind]!=test_df['Status'][j_ind]:
                checked_df['Status'][i_ind]=test_df['Status'][j_ind]
                print(checked_df['Status'][i_ind],test_df['Status'][j_ind])


test_df
filter_cols=['index','Status']
checked_df=checked_df.filter(items=filter_cols)
checked_df=checked_df.append(test_df)


In [22]:
checked_df=pd.read_excel('Checked_data.xlsx')
checked_df
#checked_df=checked_df.append(test_df)
import numpy as np
#checked_df = checked_df.drop_duplicates(subset='index', keep="first")

#checked_df.fillna('halwa')
print(str(checked_df['Task Due Date'].iloc[3])=='nan')
checked_df.shape[0]

True


44

In [20]:
def task_update(service,task_date):

    task_list=get_task(df[df['Service']==service],task_date).split('\n')
    print(task_list)
    task_list = list(filter(None, task_list))
    
    objects=[]
    for i in task_list:
        objects+=[sg.Checkbox(k,key=k)]
    col=[objects]
    check_items=[]
    for i in col:
        for j in i:
            check_items.append([j])
    return(check_items)
task_update('OPF','2020-09-16')

['Miscellanous Reports - AAM Performance Portal - PPF Load tasks', 'Miscellanous Reports - Solvency II: AllianzGI', 'Risk Reports - Risk Profile Report', '']


[[<PySimpleGUI.PySimpleGUI.Checkbox at 0x20a940ea5e0>],

In [25]:
def test3(service,date): 
    bf=df[df['Service']==service].copy()
    a=[]
    for i,j in bf[bf['Date'].isin([date])].filter(items=['Task/Service/Reports']).items():
        for k in j:
            a+=[sg.Checkbox(k,key=k)]
    col=[a]
    b=[]
    for i in col:
        for j in i:
            b.append([j])
    return(b)
test3('OPF','2020-09-16')


[[<PySimpleGUI.PySimpleGUI.Checkbox at 0x20a9422a9a0>],

In [8]:


def test2(): 
    a=[]
    for i,j in df[df['Date'].isin(['2020-09-22'])].filter(items=['Task/Service/Reports']).items():
        for k in j:
            a+=[sg.Checkbox(k,key=k)]
    col=[a]
    b=[]
    for i in col:
        for j in i:
            b.append([j])
    return(b)


In [9]:
checked_df=pd.read_excel('Checked_data.xlsx')
test_df= pd.DataFrame(values5, index=['Status']).T.reset_index()

In [13]:
checked_df.append(test_df).reset_index(drop=True)

index  Status
0         Allianz GI Hong Kong Reporting - Daily Pair    True
1                                    US Daily Reports    True
2   Miscellanous Reports - 2Sigma Reporting (ProUI...    True
3   Risk Reports - VaR – daily delivery to externa...    True
4      Risk Reports - Controlling Volatility Strategy   False
5                  Risk Reports - Dynamic Risk Parity   False
6     Risk Reports - Allianz Fundamental Strategy LUX   False
7                  Risk Reports - Cash Flow Report EU   False
8     Risk Reports - Daily Liquidity Risk Report (AP)   False
9     Risk Reports - Daily Liquidity Risk Report (US)   False
10                    Risk Reports - Merger Arbitrage   False
11          Risk Reports - Allianz Fixed Income Macro   False
12             Risk Reports - VaR-Monitoring MAS (EU)   False
13             Risk Reports - VaR-Monitoring MAS (AP)   False
14             Risk Reports - VaR-Monitoring MAS (US)   False
15                   Risk Reports - VaR-Monitoring FI   False
16              Risk Reports - Var Monitoring VR-Bank   False
17               Risk Reports - Var Monitoring Suscap   False
18  Risk Reports - VaR – weekly delivery to extern...    True
19                                  Jupiter Reporting    True
20  Miscellanous Reports - AAM Performance Portal ...    True
21      Miscellanous Reports - Solvency II: AllianzGI    True
22                 Risk Reports - Risk Profile Report    True